# 5.1 - Point set registration

This notebook shows the initial implementation of iterative closest point (ICP) registration used in this project.

Given two point clouds, $A$ and $B$, the algorithm works like this:

1. Find nearest neighbors between $A$ and $B$ and remove outliers (establish correspondence)
1. Find rotation $R$ and translation $t$ using singular value decomposition that, when applied to $A$, aligns $A$ and $B$.
1. Calculate root mean square errors between corresponding points in $A$ and $B$
1. Update $A$ and repeat from 1. until RMSE is sufficiently low.

This is however only useful when the clouds are close to one another. They are close to one another in the reliability dataset, so in that dataset we can use this approach. **TODO: Implement a simple brute force search (simple, quickly implemented) for non-close point clouds until a better approach is available.**

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches
from scipy import spatial, optimize
import logging
import sys

sys.path.insert(0, "../src")
from rsa.transformation_3d import Transformation
from rsa.series import RSA_Series_Logic
from rsa.dataset import RSA_Dataset
from rsa import registration

In [ ]:
T = Transformation.factory

A = np.array([
    [2, 4, 0],
    [2, 3, 0],
    [3, 3, 0],
    [3, 2, 0],
    [4, 2, 0],
    [4, 4, 0],
])
B = T().tl(-np.mean(A, axis=0)).rt(np.radians([0, 0, 10])).tl(np.mean(A, axis=0) + np.array([0.25, 0.25, 0])).tf(A, add_w=True)

print("A:", A)
print("B:", B)

plt.scatter(A[:, 0], A[:, 1], label="A")
plt.scatter(B[:, 0], B[:, 1], label="B")
plt.grid()
plt.axis("equal")
plt.legend()
plt.show()

In [ ]:
# NOTE: source code is documented in registration.py

# find nearest neighbors and remove outliers
def find_neighbors(A, B):
    dist_map = spatial.distance.cdist(A[:, :3], B[:, :3])
    A_idcs, B_idcs = optimize.linear_sum_assignment(dist_map)
    # TODO: add outlier removal here
    return A_idcs, B_idcs, dist_map[A_idcs, B_idcs]
 
def best_fit_transform(A, B):
    # translate to origin
    A_centroid = np.mean(A[:, :3], axis=0)
    B_centroid = np.mean(B[:, :3], axis=0)

    Ac = A[:, :3] - A_centroid
    Bc = B[:, :3] - B_centroid

    # rotation matrix
    H = np.dot(Ac.T, Bc)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
        Vt[2, :] *= -1  # TODO: verify this step
        R = np.dot(Vt.T, U.T)

    # return rotation matrix R and translation vector t
    return R, B_centroid.T - np.dot(R, A_centroid.T)

def icp(A, B, **kwargs):
    AA = np.copy(A)

    prev_error = 0
    
    for i in range(kwargs.get("iterations", 20)):

        # find neighbors
        A_idcs, B_idcs, dists = find_neighbors(AA, B)

        # find rmse
        # rmse = np.sqrt(np.mean(np.square(dists)))
        me = np.mean(dists)
        if abs(prev_error - me) < kwargs.get("tol", 0.00035):
            break

        prev_error = me

        # find best transformation
        R, t = best_fit_transform(AA[A_idcs, :3], B[B_idcs, :3])
        AA = Transformation.f().m(R, add_w=True).tl(t).tf(AA, add_w=True)

    #A_centroid = np.mean(A[AA_idcs, :3], axis=0)
    #AA = A[AA_idcs, :3] - A_centroid
    #BB = B[B_idcs, :3] - A_centroid

    R, t = best_fit_transform(A[A_idcs, :3], B[B_idcs, :3])
    return R, t, dists, i

R, t, dists, i = icp(A, B)
R, t, dists, i

In [ ]:
plt.figure(figsize=(14,10))

plt.scatter(A[:, 0], A[:, 1], marker="+", s=100, label="A")

A_R = Transformation.f().m(R, add_w=True).tf(A, add_w=True)
plt.scatter(A_R[:, 0], A_R[:, 1], marker="+", s=100, label="A_R")

A_Rt = Transformation.f().tl(t).tf(A_R)
plt.scatter(A_Rt[:, 0], A_Rt[:, 1], marker="+", s=100, label="A_Rt")

plt.scatter(B[:, 0], B[:, 1], marker="x", s=100, label="B")

plt.scatter(0, 0, marker=".", s=200, c="black")

plt.grid()
plt.axis("equal")
plt.legend(loc="lower right")
plt.show()

In [ ]:
# by moving A so that its centroid is in the global origin, we eliminate translation caused by the rotation transformation; this is useful for the application of this method

A_centroid = np.mean(A[:, :3], axis=0)

A_c = A[:, :3] - A_centroid
B_cA = B[:, :3] - A_centroid

R, t, dists, i = icp(A_c, B_cA)
print("R:", R)
print("t:", t)
print("dists:", dists)
print("i:", i)

plt.figure(figsize=(14,10))

plt.scatter(A_c[:, 0], A_c[:, 1], marker="+", s=100, label="A_c")

A_cR = Transformation.f().m(R, add_w=True).tf(A_c, add_w=True)
plt.scatter(A_cR[:, 0], A_cR[:, 1], marker="+", s=100, label="A_cR")

A_cRt = Transformation.f().tl(t).tf(A_cR)
plt.scatter(A_cRt[:, 0], A_cRt[:, 1], marker="+", s=100, label="A_cRt")

plt.scatter(B_cA[:, 0], B_cA[:, 1], marker="x", s=100, label="B_cA")

plt.scatter(0, 0, marker=".", s=200, c="black")

plt.grid()
plt.axis("equal")
plt.legend(loc="lower right")
plt.show()